In [1]:
import os

# Dictionary to hold file names and their contents
files_content_dict = {}

# Directory containing the .txt files
directory = './data/en/'

# Check if directory exists
if not os.path.exists(directory):
    print(f"The directory {directory} does not exist.")
else:
    # Loop through all the files in the directory
    for filename in os.listdir(directory):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        # Check if it is a file and has a .txt extension
        if os.path.isfile(file_path) and filename.endswith('.txt'):
            # Open and read the file content
            with open(file_path, 'r', encoding='utf-8') as file:
                # Read the contents of the file
                content = file.read()
                # Add to dictionary with the file name (without extension) as the key
                files_content_dict[os.path.splitext(filename)[0]] = content

# Printing the content dict just for demonstration purposes
for file_name, file_content in files_content_dict.items():
    print(f"{file_name}: {file_content[:50]}...")  # Print the first 50 characters of each file

# You can now use 'files_content_dict' as needed


Physiology_Levy: We are pleased that the following section authors ...
Cell_Biology_Alberts: The surface of our planet is populated by living t...
Pathoma_Husain: Growth Adaptations, Cellular Injury, and Cell Deat...
Psichiatry_DSM-5: PRESIDENT DILIP V. IESTE, M.D.

PRESIDENT-ELECT JE...
Immunology_Janeway: dendritic cells. 366 9-14 Cell-adhesion molecules ...
Anatomy_Gray: What is anatomy?

Anatomy includes those structure...
Pharmacology_Katzung: (All nonresearch use illegal under federal law.)

...
Surgery_Schwartz: Part IBasic ConsiderationsBrunicardi_Ch01_p0001-p0...
Biochemistry_Lippincott: For additional ancillary materials related to this...
Neurology_Adams: We are very pleased to bring you the 11th edition ...
First_Aid_Step2: Database of High-Yield Facts

The seventh edition ...
Obstentrics_Williams: In the olowingpages I have attempted to set orth, ...
Histology_Ross: OVERVIEW OF METHODS USED IN HISTOLOGY / 1 TISSUE P...
InternalMed_Harrison: xxxviii its related products in 

In [2]:
def split_into_para_groups(text, n = 3, max_length_group = 128*7):
    """
    Split the text into groups of n paragraphs each.
    If the length of the group exceeds max_length_group into groups of length <= max_length_group using for loop to add segments
    and NOT checking the length of the group at each step.
    because this might add a paragraph that exceeds the max_length_group.
    """
    para = text.split("\n")
    para = [p.strip() for p in para if p.strip() != ""]
    para_max_len_opt = []
    for p in para:
        if len(p) > max_length_group:
            for i in range(0, len(p), max_length_group):
                para_max_len_opt.append(p[i:i+max_length_group])
        else:
            para_max_len_opt.append(p)
    para = para_max_len_opt

    para_groups = []
    for i in range(0, len(para), n):
        para_groups.append("\n".join(para[i:i+n]))

    
    return para_groups
    


In [3]:

# create a dict of file_name and para_groups

files_para_groups_dict = {}

for file_name, file_content in files_content_dict.items():
    files_para_groups_dict[file_name] = split_into_para_groups(file_content, n = 3, max_length_group = 128*7)

In [4]:
print(files_para_groups_dict.keys())            

dict_keys(['Physiology_Levy', 'Cell_Biology_Alberts', 'Pathoma_Husain', 'Psichiatry_DSM-5', 'Immunology_Janeway', 'Anatomy_Gray', 'Pharmacology_Katzung', 'Surgery_Schwartz', 'Biochemistry_Lippincott', 'Neurology_Adams', 'First_Aid_Step2', 'Obstentrics_Williams', 'Histology_Ross', 'InternalMed_Harrison', 'Pediatrics_Nelson', 'Gynecology_Novak', 'First_Aid_Step1', 'Pathology_Robbins'])


In [5]:
files_para_groups_dict['Physiology_Levy'][:2]

['We are pleased that the following section authors have continued as members of the seventh edition team: Drs. Kalman Rubinson and Eric Lang (nervous system), Dr. James Watras (muscle), Dr. Achilles Pappano (cardiovascular system), Drs. Michelle Cloutier and Roger Thrall (respiratory system), Drs. Kim Barrett and Helen Raybould (gastrointestinal system), and Dr. Bruce White (endocrine and reproductive systems). We also welcome the following authors: Dr. Withrow Gil Wier (cardiovascular system), and Dr. John Harrison (endocrine and reproduction systems).\nAs in the previous editions of this textbook, we have attempted to emphasize broad concepts and to minimize the compilation of isolated facts. Each chapter has been written to make the text as lucid, accurate, and current as possible. We have included both clinical and molecular information in each section, as feedback on these features has indicated that this information serves to provide clinical context and new insights into physio

In [6]:
# conext bot

from vllm import LLM, SamplingParams
import vllm
import torch
from typing import List, Callable, Optional
from vllm.sampling_params import SamplingParams
from vllm.model_executor.input_metadata import InputMetadata
#base_model_id = "ehartford/dolphin-2.0-mistral-7b"
#base_model_id = "HuggingFaceH4/zephyr-7b-alpha"
#base_model_id = "amazon/MistralLite"
base_model_id = "ehartford/dolphin-2.2.1-mistral-7b"
llm = LLM(model=base_model_id)


INFO 11-10 14:27:13 llm_engine.py:72] Initializing an LLM engine with config: model='ehartford/dolphin-2.2.1-mistral-7b', tokenizer='ehartford/dolphin-2.2.1-mistral-7b', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 11-10 14:27:24 llm_engine.py:207] # GPU blocks: 1998, # CPU blocks: 2048


In [151]:
template = """<|im_start|>system
You will determine the specific chapter or section title in which the provided text from the book {book} is found. To do this, you'll be given the current text and preceding context. Please update the given information with the relevant chapter or section name, or provide a new one if the text introduces a different section. <|im_end|>
<|im_start|>user
## Previous Context
{previous_context} 
## Current Text
{text} <|im_end|>
<|im_start|>assistant
"""

# assistant_preprompt = "The contex of the current content is: "

sampling_params = SamplingParams(temperature=0.1, max_tokens=128)

In [152]:
def generate_prompt(previous_context, book, text):
    return template.format(previous_context = previous_context, book = book, text = text) + assistant_preprompt


In [153]:
def get_next_contexts(previous_contexts, books, texts):
    prompts = [generate_prompt(previous_context, book, text) for previous_context, book, text in zip(previous_contexts, books, texts)]
    outputs = llm.generate(prompts, sampling_params)
    contexts = [output.outputs[0].text for output in outputs]
    return contexts


In [154]:
# generate a random file_paras

file_paras = {}

file_paras = {file_name: files_para_groups_dict[file_name][:5] for file_name in files_para_groups_dict.keys()}


# file_paras = dict(files_para_groups_dict)



In [155]:
result_dict = {book: [] for book in file_paras.keys()}
current_active_contexts = {book: "Current section is the first section. No previous context." for book in file_paras.keys()}

i = 0
while len(current_active_contexts) > 0:
    books_active = list(current_active_contexts.keys())
    previous_contexts = [current_active_contexts[book] for book in books_active]
    texts = [file_paras[book][i] for book in books_active]
    next_contexts = get_next_contexts(previous_contexts, books_active, texts)
    for book, next_context in zip(books_active, next_contexts):
        result_dict[book].append(next_context)
        current_active_contexts[book] = next_context
    i += 1
    # remove books that have reached the end
    current_active_contexts = {book: context for book, context in current_active_contexts.items() if i <
                                len(file_paras[book])}

Processed prompts:   0%|          | 0/18 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 18/18 [00:03<00:00,  5.01it/s]


In [156]:
result_dict

for book, contexts in result_dict.items():
    print(f"Book: {book}")
    print(len(contexts))
    print("=======================================")

Book: Physiology_Levy
5
Book: Cell_Biology_Alberts
5
Book: Pathoma_Husain
5
Book: Psichiatry_DSM-5
5
Book: Immunology_Janeway
5
Book: Anatomy_Gray
5
Book: Pharmacology_Katzung
5
Book: Surgery_Schwartz
5
Book: Biochemistry_Lippincott
5
Book: Neurology_Adams
5
Book: First_Aid_Step2
5
Book: Obstentrics_Williams
5
Book: Histology_Ross
5
Book: InternalMed_Harrison
5
Book: Pediatrics_Nelson
5
Book: Gynecology_Novak
5
Book: First_Aid_Step1
5
Book: Pathology_Robbins
5


In [157]:
# save the result as csv after converting to dataframe
# would need to flatten the dict of lists 

file_names = []
para = []
context_list = []
for file_name, contexts in result_dict.items():
    for i, context in enumerate(contexts):
        file_names.append(file_name)
        para.append(file_paras[file_name][i])
        context_list.append(context)

import pandas as pd

df = pd.DataFrame({"file_name": file_names, "para": para, "context": context_list})

df.to_csv("data/contexts.csv", index = False)

df.head()

,file_name,para,context
0,Physiology_Levy,We are pleased that the following section auth...,"\n\n""We are pleased that the following section..."
1,Physiology_Levy,iscipline of physiology and are presented thro...,"\n\n""Cells need a constant supply of energy. T..."
2,Physiology_Levy,Liver: Converts precursors into fuel storage f...,\n\nChapter 1: Cellular Energy Metabolism\nSec...
3,Physiology_Levy,Adipose tissue: Stores fuel during feeding (e....,\n\nChapter 1: Cellular Energy Metabolism\nSec...
4,Physiology_Levy,Nervous system: Monitors oxygen levels and nut...,\n\nChapter 1: Cellular Energy Metabolism\nSec...


In [158]:
# print some of the contexts

for i in range(10):
    print(f"Context {i}")
    print(df.context[i])
    print("=======================================")

Context 0


"We are pleased that the following section authors have continued as members of the seventh edition team: Drs. Kalman Rubinson and Eric Lang (nervous system), Dr. James Watras (muscle), Dr. Achilles Pappano (cardiovascular system), Drs. Michelle Cloutier and Roger Thrall (respiratory system), Drs. Kim Barrett and Helen Raybould (gastrointestinal system), and Dr. Bruce White (endocrine and reproductive systems). We also welcome the following authors: Dr. Withrow Gil Wier (cardiov
Context 1


"Cells need a constant supply of energy. This energy is derived from the hydrolysis of adenosine triphosphate (ATP). If not replenished, the cellular ATP supply would be depleted in most cells in less than 1 minute. Thus, ATP must be continuously synthesized. This in turn requires a steady supply of cellular fuels. However, the cellular fuels (e.g., glucose, fatty acids, and ketoacids) are present in the blood at levels that can support cellular metabolism only for
Context 2


Chapter 1:

In [159]:
# write to a temp file

with open("data/contexts.txt", "w") as f:
    for i in range(len(df)):
        f.write(df.context[i] + "\n")